In [1]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from tests_hw4 import test_prediction, test_generation


from torch.optim import Optimizer


In [2]:
# load all that we need

dataset = np.load('../dataset/wiki.train.npy', allow_pickle=True)
devset = np.load('../dataset/wiki.valid.npy', allow_pickle=True)
fixtures_pred = np.load('../fixtures/prediction.npz')  # dev
fixtures_gen = np.load('../fixtures/generation.npy')  # dev
fixtures_pred_test = np.load('../fixtures/prediction_test.npz')  # test
fixtures_gen_test = np.load('../fixtures/generation_test.npy')  # test
vocab = np.load('../dataset/vocab.npy')

In [3]:
# data loader

class LanguageModelDataLoader(DataLoader):
    """
        TODO: Define data loader logic here
    """
    def __init__(self, dataset, batch_size, shuffle=True):
        
        self.data = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seq_len = 70.0


        
    def BPTT(self, i, x, y):
        init_bptt = self.seq_len
        bptt = self.seq_len if np.random.random() < 0.95 else self.seq_len/2
        seq_len = int(np.random.normal(bptt, 5))
        if seq_len == 0: seq_len = self.seq_len
        
        x, y = x[:,i:i+seq_len], y[:,i:i+seq_len]
        x, y = torch.tensor(x).int(), torch.LongTensor(y)
        i += seq_len

        return x, y, init_bptt, seq_len, i


    def __iter__(self):
        # concatenate your articles and build into batches
        if self.shuffle == True:
            np.random.shuffle(self.data)
        conc_article = np.concatenate(self.data)
        nbatch = conc_article.shape[0] // self.batch_size
        trimmed_data = conc_article[:nbatch*self.batch_size+1]
        inputs = trimmed_data[:-1].reshape((self.batch_size, -1))
        targets = trimmed_data[1:].reshape((self.batch_size, -1))
        i = 0
        while i < nbatch:
          x, y, init_bptt, seq_len, i = self.BPTT(i, inputs, targets)
          yield x, y, init_bptt, seq_len

        

In [4]:
# model

class LanguageModel(nn.Module):
    """
        TODO: Define your model here
    """
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        self.dropout1 = nn.Dropout(p=0.4)
        self.embedding = nn.Embedding(vocab_size, 400)
        self.dropout2 = nn.Dropout(p=0.1)
        # self.lstm = nn.LSTM(input_size=400, hidden_size=1150, num_layers=3, dropout=0.3, batch_first=True)
        self.lstm = nn.LSTM(input_size=400, hidden_size=1150, num_layers=4, batch_first=True)
        self.dropout3 = nn.Dropout(p=0.4)
        self.classification = nn.Linear(1150, vocab_size)

    def init_weight(self, model):
        # pass
        if type(model) == nn.Embedding:
          nn.init.uniform_(model.weight, a=-0.1, b=0.1)
        elif type(model) == nn.LSTM:
          for name, para in model.named_parameters():
            if 'weight' in name:
              nn.init.uniform_(para, a=-1/33.9116499156, b=1/33.9116499156)
            else: #bias
              nn.init.constant_(para, 0.0)

    def forward(self, x, hn=None):
        # Feel free to add extra arguments to forward (like an argument to pass in the hiddens)
        # dropword = self.dropout1(x)
        # embedding = self.embedding(dropword)
        # dropemb = self.dropout2(embedding)
        # output, (hn, cn) = self.lstm(embedding) if hn==None else self.lstm(embedding, hn) 
        # output = self.dropout3(output)
        # output = self.classification(output)
        
        embedding = self.embedding(x)
        output, (hn, cn) = self.lstm(embedding) if hn==None else self.lstm(embedding, hn) 
        output = self.classification(output)
        return output, (hn, cn)
        

In [5]:
# model trainer

class LanguageModelTrainer:
    def __init__(self, model, loader, max_epochs=1, run_id='exp'):
        """
            Use this class to train your model
        """
        # feel free to add any other parameters here
        self.model = model
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.run_id = run_id
        self.lr = 2e-3
        # self.lr = 30
        self.weight_decay = 5e-4
        self.alpha = 2
        self.beta = 1
        
        # TODO: Define your optimizer and criterion here
        self.optimizer = torch.optim.Adam(model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.criterion = torch.nn.CrossEntropyLoss()
        self.scaler = torch.cuda.amp.GradScaler()
    
    def train(self):
        # self.model.apply(self.model.init_weight)
        self.model.train() # set to training mode
        epoch_loss = 0
        num_batches = 0
        
        # for batch_num, (inputs, targets) in enumerate(self.loader):
        for batch_num, (x, y, init_bptt, seq_len) in enumerate(self.loader):
            x, y = x.cuda(), y.cuda()
            # self.optimizer.param_groups[0]['lr'] *= seq_len / init_bptt
            epoch_loss += self.train_batch(x, y)
        epoch_loss = epoch_loss / (batch_num + 1)
        self.epochs += 1
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)

    def train_batch(self, inputs, targets):
        """ 
            TODO: Define code for training a single batch of inputs
        
        """
        self.optimizer.zero_grad()

        with torch.cuda.amp.autocast():    
            outputs, (hn, cn) = self.model(inputs) #enforce_sorted=False)
            outputs = torch.reshape(outputs, (-1, outputs.size(2)))
            targets = torch.flatten(targets)
            loss = self.criterion(outputs, targets)
        self.scaler.scale(loss).backward() 
        # nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.25, norm_type=2)
        self.scaler.step(self.optimizer)
        self.scaler.update()
        
                
        # outputs, (hn, cn) = self.model(inputs) #enforce_sorted=False)
        # outputs = torch.reshape(outputs, (-1, outputs.size(2)))
        # targets = torch.flatten(targets)
        # loss = self.criterion(outputs, targets)
        # loss.backward()
        # nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.25, norm_type=2)
        # self.optimizer.step()

        return loss

    
    def test(self):
        # don't change these
        self.model.eval() # set to eval mode
        predictions = TestLanguageModel.prediction(fixtures_pred['inp'], self.model) # get predictions
        self.predictions.append(predictions)
        generated_logits = TestLanguageModel.generation(fixtures_gen, 10, self.model) # generated predictions for 10 words
        generated_logits_test = TestLanguageModel.generation(fixtures_gen_test, 10, self.model)
        nll = test_prediction(predictions, fixtures_pred['out'])
        generated = test_generation(fixtures_gen, generated_logits, vocab)
        generated_test = test_generation(fixtures_gen_test, generated_logits_test, vocab)
        self.val_losses.append(nll)
        
        self.generated.append(generated)
        self.generated_test.append(generated_test)
        self.generated_logits.append(generated_logits)
        self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = TestLanguageModel.prediction(fixtures_pred_test['inp'], self.model) # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, nll))
        return nll

    def save(self):
        # don't change these
        model_path = os.path.join('experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()},
            model_path)
        np.save(os.path.join('experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
        np.save(os.path.join('experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated_test[-1])


In [6]:
class TestLanguageModel:
    def prediction(inp, model, h=None):
        """
            TODO: write prediction code here
            
            :param inp:
            :return: a np.ndarray of logits
        """
        x = torch.cuda.LongTensor(inp)
        outputs, (hn, cn)= model(x) if h==None else model(x, h)
        return outputs[:,-1].cpu().detach().numpy()

        
    def generation(inp, forward, model):
        """
            TODO: write generation code here

            Generate a sequence of words given a starting sequence.
            :param inp: Initial sequence of words (batch size, length)
            :param forward: number of additional words to generate
            :return: generated words (batch size, forward)
        """   
        model.eval()
        gen_seq = []
 
        with torch.no_grad():
            x = torch.cuda.LongTensor(inp)
            for i in range(forward):
              if i == 0:
                outputs, (hn, cn) = model(x)
              else:
                outputs, (hn, cn) = model(x, (hn, cn))
              outputs = outputs[:,-1]
              word = torch.argmax(outputs, dim=1)
              word = word.cpu().detach().numpy()
              gen_seq.append(word)

        result = np.array(gen_seq).T
        return result

In [7]:
# TODO: define other hyperparameters here

# NUM_EPOCHS = 750
NUM_EPOCHS = 20
# BATCH_SIZE = 80
BATCH_SIZE = 128


In [8]:
run_id = str(int(time.time()))
if not os.path.exists('./experiments'):
    os.mkdir('./experiments')
os.mkdir('./experiments/%s' % run_id)
print("Saving models, predictions, and generated words to ./experiments/%s" % run_id)

Saving models, predictions, and generated words to ./experiments/1651507520


In [9]:
model = LanguageModel(len(vocab)).cuda()
loader = LanguageModelDataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)
trainer = LanguageModelTrainer(model=model, loader=loader, max_epochs=NUM_EPOCHS, run_id=run_id)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
best_nll = 1e30 
for epoch in range(NUM_EPOCHS):
    trainer.train()
    nll = trainer.test()
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch)+" with NLL: "+ str(best_nll))
        trainer.save()
    

TypeError: iter() returned non-iterator of type 'tuple'

In [ ]:
# Don't change these
# plot training curves
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses, label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()

In [ ]:
# see generated output
print (trainer.generated[-1]) # get last generated output